# Define Environment Variables

In [1]:
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen3:4b"
LLM_SEED = 42 
LLM_TEMPERATURE = 0.0
TEST_PROMPT0 = "What time is it?"
TEST_PROMPT1 = "What is the price of gold right now?" 
TEST_PROMPT2 = "How many 1 in 111111111111111?"

# Initialize Ollama Chatbot

In [2]:
from langchain_ollama import ChatOllama

# Set up the Ollama chat model with specified LLM model and parameters
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    seed=LLM_SEED,
    stream=True
)

# Define Tools

1. **Python Tool:** This tool allows the chatbot to execute Python code, enabling it to perform calculations, data processing, and other tasks that require programming logic.

2. **Commodity Price Tool:** This tool fetches real-time commodity prices from the `API Ninjas` API, allowing the chatbot to provide up-to-date information on various commodities.
    - Go to [``API Ninjas``](https://api-ninjas.com/)
    - Click the "Sign Up" button
    - Create an account
    - Log in
    - Click the "My Account" button
    - Click the "Show API Key" button

    In the next cell, you will need to:
    - Replace the content of the variable `NINJA_API_KEY` with your API key.
    - Run the cell to test the API tool

In [3]:
from langchain_core.tools import tool, Tool
from typing import Annotated, List
import io
import contextlib
import requests

NINJA_API_KEY = "MdaCcVUHseixkwpNGhumWg==9mb676ZThl4du9aP"

@tool
def execute_python(py_code: Annotated[str, "Python code to execute"]) -> str:
    """Executes a Python code and returns its standard output (you have to use the print() function)."""
    output = io.StringIO()
    try:
        with contextlib.redirect_stdout(output):
            exec(py_code, {})
        return output.getvalue().strip() or "Code executed with no output."
    except Exception as e:
        return f"Error: {str(e)}"

# Test the tool
print(execute_python.invoke("print('Hello, World!')"))

@tool
def get_commodity_price(
    commodity: Annotated[str, "The name of the commodity to get the price for"]
) -> str:
    """Get the current price of a commodity using the Ninja API."""
    api_url = 'https://api.api-ninjas.com/v1/commodityprice?name={}'.format(commodity)
    response = requests.get(api_url, headers={'X-Api-Key': NINJA_API_KEY})
    if response.status_code == requests.codes.ok:
        return response.text
    else:
        return f"Error {response.status_code}: {response.text}"

#Test the tool
print(get_commodity_price.invoke("gold"))
    
tools = [execute_python, get_commodity_price]

Hello, World!
{"exchange": "CME", "name": "Gold Futures", "price": 3409.1, "updated": 1749737826}


# Prepare Reason-and-Act (ReAct) Instructions

ReAct (Reason-and-Act) prompting combines:

- **Reasoning:** The model explains its thinking process.
- **Acting:** The model chooses and performs actions (like using a calculator, web search, or database query).

The prompt encourages the model to alternate between these two steps, creating a loop:

→ Think → Act → Observe → Think → Act → … until the task is complete.

**Reference:**

(Yao et al., 2023) Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2023, January). ReAct: Synergizing reasoning and acting in language models. In *International Conference on Learning Representations (ICLR)*.]{https://doi.org/10.48550/arXiv.2210.03629}

In [ ]:
from inspect import signature

tools_descriptions = "\n- ".join([f"{tool.name}{signature(tool.func)} - {tool.description}" for tool in tools]) 

# Taken from https://smith.langchain.com/hub/hwchase17/react-json
instructions = f"""You are a reasoning and acting assistant. You solve complex tasks by thinking step-by-step and using tools when needed. Follow this format exactly:

Question: (The user's question or task.)

Thought: (Think about what to do next.)

Action: (What tool to use, if any, and what input to provide. Use the exact JSON format below.)
```
{{
  "action": $TOOL_NAME,
  "action_input": $TOOL_INPUT
}}
```

Observation: (What was the result of the action.)

... (Repeat Thought → Action → Observation as needed)

Final Answer: (Your final response to the user.)

Important Rules:
- Always begin with a Thought before taking an action.
- If an action is needed, use the following format:
  Action: 
  ```
  {{
    "action": "tool_name",
    "action_input": "your input here"
  }}
  ```
- If no action is needed, give your final answer using the following format:
  Final Answer: <your final answer here>

Available Tools:
- {tools_descriptions}

Think clearly. Be concise. Don't skip steps.
"""


print(instructions)

You are a reasoning and acting assistant. You solve complex tasks by thinking step-by-step and using tools when needed. Follow this format exactly:

Question: (The user's question or task.)

Thought: (Think about what to do next.)

Action: (What tool to use, if any, and what input to provide. Use the exact JSON format below.)
```
{
  "action": $TOOL_NAME,
  "action_input": $TOOL_INPUT
}
```

Observation: (What was the result of the action.)

... (Repeat Thought → Action → Observation as needed)

Final Answer: (Your final response to the user.)

Important Rules:
- Always begin with a Thought before taking an action.
- If an action is needed, use the following format:
  Action: 
  ```
  {
    "action": "tool_name",
    "action_input": "your input here"
  }
  ```
- If you want to give your final answer, use the following format:
  Final Answer: 
  ```
  your final answer here
  ```

Available Tools:
- execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Py

# Testing the Agent

In [7]:
from src.Agent import Agent
from langchain_core.messages import SystemMessage

# Create a chat history with a system message and a human message
agent = Agent(llm=llm, tools=tools, history=[SystemMessage(content=instructions)])
agent.invoke(TEST_PROMPT0)
#agent.invoke(TEST_PROMPT1)
# agent.invoke(TEST_PROMPT2)

================================ System Message ================================

You are a reasoning and acting assistant. You solve complex tasks by thinking step-by-step and using tools when needed. Follow this format exactly:

Question: (The user's question or task.)

Thought: (Think about what to do next.)

Action: (What tool to use, if any, and what input to provide. Use the exact JSON format below.)
```
{
  "action": $TOOL_NAME,
  "action_input": $TOOL_INPUT
}
```

Observation: (What was the result of the action.)

... (Repeat Thought → Action → Observation as needed)

Final Answer: (Your final response to the user.)

Important Rules:
- Always begin with a Thought before taking an action.
- If an action is needed, use the following format:
  Action: 
  ```
  {
    "action": "tool_name",
    "action_input": "your input here"
  }
  ```
- If you want to give your final answer, use the following format:
  Final Answer: 
  ```
  your final answer here
  ```

Available Tools:
- execut